# Data Aggregation and Group Operations

In [1]:
import numpy as np
import pandas as pd
PREVIOUS_MAX_ROWS = pd.options.display.max_rows
pd.options.display.max_rows = 20
np.random.seed(12345)
import matplotlib.pyplot as plt
plt.rc('figure', figsize=(10, 6))
np.set_printoptions(precision=4, suppress=True)

## GroupBy Mechanics

In [2]:
df = pd.DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'],
                   'key2' : ['one', 'two', 'one', 'two', 'one'],
                   'data1' : np.arange(5),
                   'data2' : np.arange(5)})
df

,key1,key2,data1,data2
0,a,one,0,0
1,a,two,1,1
2,b,one,2,2
3,b,two,3,3
4,a,one,4,4


In [3]:
# 通过series来分组
grouped = df['data1'].groupby(df['key1'])
print(type(grouped))
print(grouped.sum())
print(grouped.mean())

<class 'pandas.core.groupby.generic.SeriesGroupBy'>
key1
a    5
b    5
Name: data1, dtype: int32
key1
a    1.666667
b    2.500000
Name: data1, dtype: float64


In [4]:
grouped.mean()

key1
a    1.666667
b    2.500000
Name: data1, dtype: float64

In [5]:
means = df['data1'].groupby([df['key1'], df['key2']]).mean()
means

key1  key2
a     one     2
      two     1
b     one     2
      two     3
Name: data1, dtype: int32

In [6]:
means.unstack()

key2,one,two
key1,,
a,2,1
b,2,3


In [7]:
states = np.array(['Ohio', 'California', 'California', 'Ohio', 'Ohio'])
years = np.array([2005, 2005, 2006, 2005, 2006])
df2 = pd.DataFrame({'state': states, 'year': years, 'data': df['data1']})
df2
# [0, 1, 2, 3, 4]

,state,year,data
0,Ohio,2005,0
1,California,2005,1
2,California,2006,2
3,Ohio,2005,3
4,Ohio,2006,4


In [8]:
# 通过ndarray来分组
df['data1'].groupby([states, years]).mean()

California  2005    1.0
            2006    2.0
Ohio        2005    1.5
            2006    4.0
Name: data1, dtype: float64

In [9]:
df.groupby('key1').mean()
df.groupby(['key1', 'key2']).mean()

data1  data2
key1 key2              
a    one       2      2
     two       1      1
b    one       2      2
     two       3      3

In [10]:
df.groupby(['key1', 'key2']).size()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

In [11]:
df = pd.DataFrame({'Animal': ['Falcon', 'Falcon',
                              'Parrot', 'Parrot'],
                   'Max Speed': [380., 370., 24., 26.]})
print(df)
group = df.groupby(['Animal'])
print(group.mean())
print()
print(group.groups) # dict {group name -> group labels}

   Animal  Max Speed
0  Falcon      380.0
1  Falcon      370.0
2  Parrot       24.0
3  Parrot       26.0
        Max Speed
Animal           
Falcon      375.0
Parrot       25.0

{'Falcon': [0, 1], 'Parrot': [2, 3]}


In [12]:
# 多级索引的group by, 可以指定level参数
>>> arrays = [['Falcon', 'Falcon', 'Parrot', 'Parrot'],
...           ['Captive', 'Wild', 'Captive', 'Wild']]
>>> index = pd.MultiIndex.from_arrays(arrays, names=('Animal', 'Type'))
>>> df = pd.DataFrame({'Max Speed': [390., 350., 30., 20.]},
...                   index=index)
>>> print(df)
>>> print()
# >>> print(df.groupby(['Animal']).mean())
# >>> print()
# >>> print(df.groupby(['Type']).mean())
# >>> print()
>>> print(df.groupby(level=0).mean())
>>> print()
>>> print(df.groupby(level='Type').mean())

                Max Speed
Animal Type              
Falcon Captive      390.0
       Wild         350.0
Parrot Captive       30.0
       Wild          20.0

        Max Speed
Animal           
Falcon      370.0
Parrot       25.0

         Max Speed
Type              
Captive      210.0
Wild         185.0


In [13]:
# dropna
l = [[1, 2, 3], [1, None, 4], [2, 1, 3], [1, 2, 2]]
df = pd.DataFrame(l, columns=["a", "b", "c"])
print(df)
print()
print(df.groupby(by=["b"]).sum())
print()
print(df.groupby(by=["b"], dropna=False).sum())

   a    b  c
0  1  2.0  3
1  1  NaN  4
2  2  1.0  3
3  1  2.0  2

     a  c
b        
1.0  2  3
2.0  2  5

     a  c
b        
1.0  2  3
2.0  2  5
NaN  1  4


In [14]:
l = [["a", 12, 12], [None, 12.3, 33.], ["b", 12.3, 123], ["a", 1, 1]]
df = pd.DataFrame(l, columns=["a", "b", "c"])
print(df)
print(df.groupby(by="a").sum())
print(df.groupby(by="a", dropna=False).sum())


      a     b      c
0     a  12.0   12.0
1  None  12.3   33.0
2     b  12.3  123.0
3     a   1.0    1.0
      b      c
a             
a  13.0   13.0
b  12.3  123.0
        b      c
a               
a    13.0   13.0
b    12.3  123.0
NaN  12.3   33.0


### Iterating Over Groups

In [15]:
df = pd.DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'],
                   'key2' : ['one', 'two', 'one', 'two', 'one'],
                   'data1' : np.arange(5),
                   'data2' : np.arange(5)})

print(df)

for name, group in df.groupby('key1'):
    print(name)         # key
    print(type(group))  # 被key切分的data frame
    print(group)

  key1 key2  data1  data2
0    a  one      0      0
1    a  two      1      1
2    b  one      2      2
3    b  two      3      3
4    a  one      4      4
a
<class 'pandas.core.frame.DataFrame'>
  key1 key2  data1  data2
0    a  one      0      0
1    a  two      1      1
4    a  one      4      4
b
<class 'pandas.core.frame.DataFrame'>
  key1 key2  data1  data2
2    b  one      2      2
3    b  two      3      3


In [16]:
for (k1, k2), group in df.groupby(['key1', 'key2']):
    print((k1, k2))
    print(group)

('a', 'one')
  key1 key2  data1  data2
0    a  one      0      0
4    a  one      4      4
('a', 'two')
  key1 key2  data1  data2
1    a  two      1      1
('b', 'one')
  key1 key2  data1  data2
2    b  one      2      2
('b', 'two')
  key1 key2  data1  data2
3    b  two      3      3


In [17]:
pieces = dict(list(df.groupby('key1')))
print(pieces)
print()
pieces['b']

{'a':   key1 key2  data1  data2
0    a  one      0      0
1    a  two      1      1
4    a  one      4      4, 'b':   key1 key2  data1  data2
2    b  one      2      2
3    b  two      3      3}



,key1,key2,data1,data2
2,b,one,2,2
3,b,two,3,3


In [18]:
list(df.groupby('key1'))

[('a',
    key1 key2  data1  data2
  0    a  one      0      0
  1    a  two      1      1
  4    a  one      4      4),
 ('b',
    key1 key2  data1  data2
  2    b  one      2      2
  3    b  two      3      3)]

In [19]:
print(df)
grouped = df.groupby(df.dtypes, axis=1)

print(df.dtypes)       # int32, object
print()
print(list(grouped))   # 按照int32和object将数据分类

  key1 key2  data1  data2
0    a  one      0      0
1    a  two      1      1
2    b  one      2      2
3    b  two      3      3
4    a  one      4      4
key1     object
key2     object
data1     int32
data2     int32
dtype: object

[(dtype('int32'),    data1  data2
0      0      0
1      1      1
2      2      2
3      3      3
4      4      4), (dtype('O'),   key1 key2
0    a  one
1    a  two
2    b  one
3    b  two
4    a  one)]


In [20]:
for dtype, group in grouped:
    print(dtype)
    print(group)

int32
   data1  data2
0      0      0
1      1      1
2      2      2
3      3      3
4      4      4
object
  key1 key2
0    a  one
1    a  two
2    b  one
3    b  two
4    a  one


In [21]:
g = df.groupby(['key1', 'key2'], axis=1)
df.groupby?
print(list(g))

[]


Signature:
df.groupby(
    by=None,
    axis=0,
    level=None,
    as_index: 'bool' = True,
    sort: 'bool' = True,
    group_keys: 'bool' = True,
    squeeze: 'bool' = <object object at 0x000001B67F42F400>,
    observed: 'bool' = False,
    dropna: 'bool' = True,
) -> 'DataFrameGroupBy'
Docstring:
Group DataFrame using a mapper or by a Series of columns.

A groupby operation involves some combination of splitting the
object, applying a function, and combining the results. This can be
used to group large amounts of data and compute operations on these
groups.

Parameters
----------
by : mapping, function, label, or list of labels
    Used to determine the groups for the groupby.
    If ``by`` is a function, it's called on each value of the object's
    index. If a dict or Series is passed, the Series or dict VALUES
    will be used to determine the groups (the Series' values are first
    aligned; see ``.align()`` method). If an ndarray is passed, the
    values are used as-is to det

### Selecting a Column or Subset of Columns

In [22]:
print(df)
print()
print(list(df.groupby('key1')['data1']))

# df.groupby('key1')[['data2']]

  key1 key2  data1  data2
0    a  one      0      0
1    a  two      1      1
2    b  one      2      2
3    b  two      3      3
4    a  one      4      4

[('a', 0    0
1    1
4    4
Name: data1, dtype: int32), ('b', 2    2
3    3
Name: data1, dtype: int32)]


In [23]:
df['data1'].groupby(df['key1'])
df[['data2']].groupby(df['key1'])

In [24]:
df.groupby(['key1', 'key2'])[['data2']].count() # 返回结果为dataframe

data2
key1 key2       
a    one       2
     two       1
b    one       1
     two       1

In [25]:
s_grouped = df.groupby(['key1', 'key2'])['data2'] # 返回结果为series
s_grouped
s_grouped.count()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
Name: data2, dtype: int64

### Grouping with Dicts and Series

In [26]:
people = pd.DataFrame(np.random.randn(5, 5),
                      columns=['a', 'b', 'c', 'd', 'e'],
                      index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])
people.iloc[2:3, [1, 2]] = np.nan # Add a few NA values
people

,a,b,c,d,e
Joe,-0.204708,0.478943,-0.519439,-0.555730,1.965781
Steve,1.393406,0.092908,0.281746,0.769023,1.246435
Wes,1.007189,NaN,NaN,0.228913,1.352917
Jim,0.886429,-2.001637,-0.371843,1.669025,-0.438570
Travis,-0.539741,0.476985,3.248944,-1.021228,-0.577087


In [27]:
mapping = {'a': 'red', 'b': 'red', 'c': 'blue',
           'd': 'blue', 'e': 'red', 'f' : 'orange'}

In [28]:
by_column = people.groupby(mapping, axis=1)
print(list(by_column))
by_column.sum()

[('blue',                c         d
Joe    -0.519439 -0.555730
Steve   0.281746  0.769023
Wes          NaN  0.228913
Jim    -0.371843  1.669025
Travis  3.248944 -1.021228), ('red',                a         b         e
Joe    -0.204708  0.478943  1.965781
Steve   1.393406  0.092908  1.246435
Wes     1.007189       NaN  1.352917
Jim     0.886429 -2.001637 -0.438570
Travis -0.539741  0.476985 -0.577087)]


,blue,red
Joe,-1.075169,2.240016
Steve,1.050769,2.732748
Wes,0.228913,2.360106
Jim,1.297183,-1.553778
Travis,2.227716,-0.639844


In [29]:
map_series = pd.Series(mapping)
map_series
people.groupby(map_series, axis=1).count()

,blue,red
Joe,2,3
Steve,2,3
Wes,1,2
Jim,2,3
Travis,2,3


### Grouping with Functions

In [30]:
people.groupby(len).sum()

,a,b,c,d,e
3,1.688911,-1.522694,-0.891281,1.342208,2.880128
5,1.393406,0.092908,0.281746,0.769023,1.246435
6,-0.539741,0.476985,3.248944,-1.021228,-0.577087


In [31]:
key_list = ['one', 'one', 'one', 'two', 'two']
people.groupby([len, key_list]).min()

a         b         c         d         e
3 one -0.204708  0.478943 -0.519439 -0.555730  1.352917
  two  0.886429 -2.001637 -0.371843  1.669025 -0.438570
5 one  1.393406  0.092908  0.281746  0.769023  1.246435
6 two -0.539741  0.476985  3.248944 -1.021228 -0.577087

### Grouping by Index Levels

In [32]:
columns = pd.MultiIndex.from_arrays([['US', 'US', 'US', 'JP', 'JP'],
                                    [1, 3, 5, 1, 3]],
                                    names=['cty', 'tenor'])
hier_df = pd.DataFrame(np.random.randn(4, 5), columns=columns)
hier_df

cty          US                            JP          
tenor         1         3         5         1         3
0      0.124121  0.302614  0.523772  0.000940  1.343810
1     -0.713544 -0.831154 -2.370232 -1.860761 -0.860757
2      0.560145 -1.265934  0.119827 -1.063512  0.332883
3     -2.359419 -0.199543 -1.541996 -0.970736 -1.307030

In [33]:
hier_df.groupby(level='cty', axis=1).count()

cty,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


## Data Aggregation

In [34]:
df
grouped = df.groupby('key1')
grouped['data1'].quantile(0.9)

key1
a    3.4
b    2.9
Name: data1, dtype: float64

In [35]:
def peak_to_peak(arr):
    return arr.max() - arr.min()
grouped.agg(peak_to_peak)

,data1,data2
key1,,
a,4,4
b,1,1


In [36]:
grouped.describe()

data1                                                data2            \
     count      mean       std  min   25%  50%   75%  max count      mean   
key1                                                                        
a      3.0  1.666667  2.081666  0.0  0.50  1.0  2.50  4.0   3.0  1.666667   
b      2.0  2.500000  0.707107  2.0  2.25  2.5  2.75  3.0   2.0  2.500000   

                                           
           std  min   25%  50%   75%  max  
key1                                       
a     2.081666  0.0  0.50  1.0  2.50  4.0  
b     0.707107  2.0  2.25  2.5  2.75  3.0

### Column-Wise and Multiple Function Application

In [37]:
tips = pd.read_csv('examples/tips.csv')
# Add tip percentage of total bill
tips['tip_pct'] = tips['tip'] / tips['total_bill']
tips[:6]

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
5,25.29,4.71,No,Sun,Dinner,4,0.186240


In [38]:
grouped = tips.groupby(['day', 'smoker'])

In [39]:
grouped_pct = grouped['tip_pct']
grouped_pct.agg('mean')

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

In [40]:
grouped_pct.agg(['mean', 'std', peak_to_peak])

mean       std  peak_to_peak
day  smoker                                  
Fri  No      0.151650  0.028123      0.067349
     Yes     0.174783  0.051293      0.159925
Sat  No      0.158048  0.039767      0.235193
     Yes     0.147906  0.061375      0.290095
Sun  No      0.160113  0.042347      0.193226
     Yes     0.187250  0.154134      0.644685
Thur No      0.160298  0.038774      0.193350
     Yes     0.163863  0.039389      0.151240

In [41]:
grouped_pct.agg([('foo', 'mean'), ('bar', np.std)])

foo       bar
day  smoker                    
Fri  No      0.151650  0.028123
     Yes     0.174783  0.051293
Sat  No      0.158048  0.039767
     Yes     0.147906  0.061375
Sun  No      0.160113  0.042347
     Yes     0.187250  0.154134
Thur No      0.160298  0.038774
     Yes     0.163863  0.039389

In [42]:
functions = ['count', 'mean', 'max']
result = grouped['tip_pct', 'total_bill'].agg(functions)
result

<ipython-input-42-94f2b21cf8d2>:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  result = grouped['tip_pct', 'total_bill'].agg(functions)


tip_pct                     total_bill                  
              count      mean       max      count       mean    max
day  smoker                                                         
Fri  No           4  0.151650  0.187735          4  18.420000  22.75
     Yes         15  0.174783  0.263480         15  16.813333  40.17
Sat  No          45  0.158048  0.291990         45  19.661778  48.33
     Yes         42  0.147906  0.325733         42  21.276667  50.81
Sun  No          57  0.160113  0.252672         57  20.506667  48.17
     Yes         19  0.187250  0.710345         19  24.120000  45.35
Thur No          45  0.160298  0.266312         45  17.113111  41.19
     Yes         17  0.163863  0.241255         17  19.190588  43.11

In [43]:
result['tip_pct']

count      mean       max
day  smoker                           
Fri  No          4  0.151650  0.187735
     Yes        15  0.174783  0.263480
Sat  No         45  0.158048  0.291990
     Yes        42  0.147906  0.325733
Sun  No         57  0.160113  0.252672
     Yes        19  0.187250  0.710345
Thur No         45  0.160298  0.266312
     Yes        17  0.163863  0.241255

In [44]:
ftuples = [('Durchschnitt', 'mean'), ('Abweichung', np.var)]
grouped['tip_pct', 'total_bill'].agg(ftuples)

<ipython-input-44-417b40324039>:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  grouped['tip_pct', 'total_bill'].agg(ftuples)


tip_pct              total_bill            
            Durchschnitt Abweichung Durchschnitt  Abweichung
day  smoker                                                 
Fri  No         0.151650   0.000791    18.420000   25.596333
     Yes        0.174783   0.002631    16.813333   82.562438
Sat  No         0.158048   0.001581    19.661778   79.908965
     Yes        0.147906   0.003767    21.276667  101.387535
Sun  No         0.160113   0.001793    20.506667   66.099980
     Yes        0.187250   0.023757    24.120000  109.046044
Thur No         0.160298   0.001503    17.113111   59.625081
     Yes        0.163863   0.001551    19.190588   69.808518

In [45]:
grouped.agg({'tip' : np.max, 'size' : 'sum'})
grouped.agg({'tip_pct' : ['min', 'max', 'mean', 'std'],
             'size' : 'sum'})

tip_pct                               size
                  min       max      mean       std  sum
day  smoker                                             
Fri  No      0.120385  0.187735  0.151650  0.028123    9
     Yes     0.103555  0.263480  0.174783  0.051293   31
Sat  No      0.056797  0.291990  0.158048  0.039767  115
     Yes     0.035638  0.325733  0.147906  0.061375  104
Sun  No      0.059447  0.252672  0.160113  0.042347  167
     Yes     0.065660  0.710345  0.187250  0.154134   49
Thur No      0.072961  0.266312  0.160298  0.038774  112
     Yes     0.090014  0.241255  0.163863  0.039389   40

### Returning Aggregated Data Without Row Indexes

In [46]:
tips.groupby(['day', 'smoker'], as_index=False).mean()

,day,smoker,total_bill,tip,size,tip_pct
0,Fri,No,18.420000,2.812500,2.250000,0.151650
1,Fri,Yes,16.813333,2.714000,2.066667,0.174783
2,Sat,No,19.661778,3.102889,2.555556,0.158048
3,Sat,Yes,21.276667,2.875476,2.476190,0.147906
4,Sun,No,20.506667,3.167895,2.929825,0.160113
5,Sun,Yes,24.120000,3.516842,2.578947,0.187250
6,Thur,No,17.113111,2.673778,2.488889,0.160298
7,Thur,Yes,19.190588,3.030000,2.352941,0.163863


## Apply: General split-apply-combine

In [47]:
def top(df, n=5, column='tip_pct'):
    return df.sort_values(by=column)[-n:]
top(tips, n=6)

,total_bill,tip,smoker,day,time,size,tip_pct
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
232,11.61,3.39,No,Sat,Dinner,2,0.291990
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


In [48]:
tips.groupby('smoker').apply(top)

total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     88        24.71  5.85     No  Thur   Lunch     2  0.236746
       185       20.69  5.00     No   Sun  Dinner     5  0.241663
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       232       11.61  3.39     No   Sat  Dinner     2  0.291990
Yes    109       14.31  4.00    Yes   Sat  Dinner     2  0.279525
       183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       172        7.25  5.15    Yes   Sun  Dinner     2  0.710345

In [49]:
tips.groupby(['smoker', 'day']).apply(top, n=1, column='total_bill')

total_bill    tip smoker   day    time  size   tip_pct
smoker day                                                             
No     Fri  94        22.75   3.25     No   Fri  Dinner     2  0.142857
       Sat  212       48.33   9.00     No   Sat  Dinner     4  0.186220
       Sun  156       48.17   5.00     No   Sun  Dinner     6  0.103799
       Thur 142       41.19   5.00     No  Thur   Lunch     5  0.121389
Yes    Fri  95        40.17   4.73    Yes   Fri  Dinner     4  0.117750
       Sat  170       50.81  10.00    Yes   Sat  Dinner     3  0.196812
       Sun  182       45.35   3.50    Yes   Sun  Dinner     3  0.077178
       Thur 197       43.11   5.00    Yes  Thur   Lunch     4  0.115982

In [50]:
result = tips.groupby('smoker')['tip_pct'].describe()
result
result.unstack('smoker')

       smoker
count  No        151.000000
       Yes        93.000000
mean   No          0.159328
       Yes         0.163196
std    No          0.039910
       Yes         0.085119
min    No          0.056797
       Yes         0.035638
25%    No          0.136906
       Yes         0.106771
50%    No          0.155625
       Yes         0.153846
75%    No          0.185014
       Yes         0.195059
max    No          0.291990
       Yes         0.710345
dtype: float64

f = lambda x: x.describe()
grouped.apply(f)

### Suppressing the Group Keys

In [51]:
tips.groupby('smoker', group_keys=False).apply(top)

,total_bill,tip,smoker,day,time,size,tip_pct
88,24.71,5.85,No,Thur,Lunch,2,0.236746
185,20.69,5.00,No,Sun,Dinner,5,0.241663
51,10.29,2.60,No,Sun,Dinner,2,0.252672
149,7.51,2.00,No,Thur,Lunch,2,0.266312
232,11.61,3.39,No,Sat,Dinner,2,0.291990
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


### Quantile and Bucket Analysis

In [52]:
frame = pd.DataFrame({'data1': np.random.randn(1000),
                      'data2': np.random.randn(1000)})
quartiles = pd.cut(frame.data1, 4)
quartiles[:10]

0    (-1.23, 0.489]
1    (-1.23, 0.489]
2    (-1.23, 0.489]
3    (-1.23, 0.489]
4    (0.489, 2.208]
5    (-1.23, 0.489]
6    (-1.23, 0.489]
7    (-1.23, 0.489]
8    (0.489, 2.208]
9    (0.489, 2.208]
Name: data1, dtype: category
Categories (4, interval[float64]): [(-2.956, -1.23] < (-1.23, 0.489] < (0.489, 2.208] < (2.208, 3.928]]

In [53]:
def get_stats(group):
    return {'min': group.min(), 'max': group.max(),
            'count': group.count(), 'mean': group.mean()}
grouped = frame.data2.groupby(quartiles)
grouped.apply(get_stats).unstack()

,min,max,count,mean
data1,,,,
"(-2.956, -1.23]",-3.399312,1.670835,92.0,-0.047129
"(-1.23, 0.489]",-2.989741,3.260383,598.0,-0.005543
"(0.489, 2.208]",-3.745356,2.954439,300.0,0.076026
"(2.208, 3.928]",-1.929776,1.765640,10.0,0.024750


In [54]:
# Return quantile numbers
grouping = pd.qcut(frame.data1, 10, labels=False)
grouped = frame.data2.groupby(grouping)
grouped.apply(get_stats).unstack()

,min,max,count,mean
data1,,,,
0,-3.399312,1.670835,100.0,-0.034083
1,-1.950098,2.628441,100.0,0.006224
2,-2.925113,2.527939,100.0,-0.098459
3,-2.315555,3.260383,100.0,0.044717
4,-2.041696,2.074345,100.0,-0.024500
5,-2.989741,2.184810,100.0,0.005278
6,-2.084231,2.458842,100.0,-0.048812
7,-3.056990,2.954439,100.0,-0.015866
8,-3.745356,2.735527,100.0,0.124168


### Example: Filling Missing Values with Group-Specific       Values

In [55]:
s = pd.Series(np.random.randn(6))
s[::2] = np.nan
s.fillna(s.mean())

0   -0.203707
1   -0.438053
2   -0.203707
3    0.401587
4   -0.203707
5   -0.574654
dtype: float64

In [56]:
states = ['Ohio', 'New York', 'Vermont', 'Florida',
          'Oregon', 'Nevada', 'California', 'Idaho']
group_key = ['East'] * 4 + ['West'] * 4
data = pd.Series(np.random.randn(8), index=states)
data

Ohio          0.786210
New York     -1.393822
Vermont       0.359262
Florida       1.170900
Oregon        0.678661
Nevada       -0.125921
California    0.150581
Idaho        -0.884475
dtype: float64

In [57]:
data[['Vermont', 'Nevada', 'Idaho']] = np.nan
print(list(data.groupby(group_key)))
data.groupby(group_key).mean()

[('East', Ohio        0.786210
New York   -1.393822
Vermont          NaN
Florida     1.170900
dtype: float64), ('West', Oregon        0.678661
Nevada             NaN
California    0.150581
Idaho              NaN
dtype: float64)]


East    0.187763
West    0.414621
dtype: float64

In [58]:
g = data.groupby(group_key)
g?
for k, v in g:
    print(k)
    print(v)

East
Ohio        0.786210
New York   -1.393822
Vermont          NaN
Florida     1.170900
dtype: float64
West
Oregon        0.678661
Nevada             NaN
California    0.150581
Idaho              NaN
dtype: float64


Type:            SeriesGroupBy
String form:     <pandas.core.groupby.generic.SeriesGroupBy object at 0x000001B68A065FA0>
Length:          2
File:            c:\python39\lib\site-packages\pandas\core\groupby\generic.py
Docstring:       <no docstring>
Class docstring:
Class for grouping and aggregating relational data.

See aggregate, transform, and apply functions on this object.

It's easiest to use obj.groupby(...) to use GroupBy, but you can also do:

::

    grouped = groupby(obj, ...)

Parameters
----------
obj : pandas object
axis : int, default 0
level : int, default None
    Level of MultiIndex
groupings : list of Grouping objects
    Most users should ignore this
exclusions : array-like, optional
    List of columns to exclude
name : str
    Most users should ignore this

Returns
-------
**Attributes**
groups : dict
    {group name -> group labels}
len(grouped) : int
    Number of groups

Notes
-----
After grouping, see aggregate, apply, and transform functions. Here are
some o

In [59]:
fill_mean = lambda g: g.fillna(g.mean())
data.groupby(group_key).apply(fill_mean)

Ohio          0.786210
New York     -1.393822
Vermont       0.187763
Florida       1.170900
Oregon        0.678661
Nevada        0.414621
California    0.150581
Idaho         0.414621
dtype: float64

In [60]:
fill_values = {'East': 0.5, 'West': -1}
# fill_func = lambda g: g.fillna(fill_values[g.name])
def fill_func(g):
    print('---------------------')
    print(f'apply g, {type(g)}:\n {g}')
    return g.fillna(fill_values[g.name])

print(group_key)
data.groupby(group_key).apply(fill_func)

['East', 'East', 'East', 'East', 'West', 'West', 'West', 'West']
---------------------
apply g, <class 'pandas.core.series.Series'>:
 Ohio        0.786210
New York   -1.393822
Vermont          NaN
Florida     1.170900
Name: East, dtype: float64
---------------------
apply g, <class 'pandas.core.series.Series'>:
 Oregon        0.678661
Nevada             NaN
California    0.150581
Idaho              NaN
Name: West, dtype: float64


Ohio          0.786210
New York     -1.393822
Vermont       0.500000
Florida       1.170900
Oregon        0.678661
Nevada       -1.000000
California    0.150581
Idaho        -1.000000
dtype: float64

### Example: Random Sampling and Permutation

In [61]:
# Hearts, Spades, Clubs, Diamonds
suits = ['H', 'S', 'C', 'D']
card_val = (list(range(1, 11)) + [10] * 3) * 4
base_names = ['A'] + list(range(2, 11)) + ['J', 'K', 'Q']
cards = []
for suit in ['H', 'S', 'C', 'D']:
    cards.extend(str(num) + suit for num in base_names)

print(cards)
print(card_val)
deck = pd.Series(card_val, index=cards)
# print(deck)

['AH', '2H', '3H', '4H', '5H', '6H', '7H', '8H', '9H', '10H', 'JH', 'KH', 'QH', 'AS', '2S', '3S', '4S', '5S', '6S', '7S', '8S', '9S', '10S', 'JS', 'KS', 'QS', 'AC', '2C', '3C', '4C', '5C', '6C', '7C', '8C', '9C', '10C', 'JC', 'KC', 'QC', 'AD', '2D', '3D', '4D', '5D', '6D', '7D', '8D', '9D', '10D', 'JD', 'KD', 'QD']
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 10, 10, 10, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 10, 10, 10, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 10, 10, 10, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 10, 10, 10]


In [62]:
deck[:13]

AH      1
2H      2
3H      3
4H      4
5H      5
6H      6
7H      7
8H      8
9H      9
10H    10
JH     10
KH     10
QH     10
dtype: int64

In [63]:
def draw(deck, n=5):
    return deck.sample(n)
draw(deck)

2D     2
JH    10
4S     4
KD    10
3D     3
dtype: int64

In [64]:
# get_suit = lambda card: card[-1] # last letter is suit
def get_suit(card):
    print(card[-1])
    return card[-1]
print([s[-1] for s in deck.index])

# deck.groupby(get_suit).apply(draw, n=2)
# groupby传递的是一个各个标签的数组， 长度和待分组数据一致
list(deck.groupby([s[-1] for s in deck.index]))
# list(deck.groupby(deck.index))

['H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D']


[('C',
  AC      1
  2C      2
  3C      3
  4C      4
  5C      5
  6C      6
  7C      7
  8C      8
  9C      9
  10C    10
  JC     10
  KC     10
  QC     10
  dtype: int64),
 ('D',
  AD      1
  2D      2
  3D      3
  4D      4
  5D      5
  6D      6
  7D      7
  8D      8
  9D      9
  10D    10
  JD     10
  KD     10
  QD     10
  dtype: int64),
 ('H',
  AH      1
  2H      2
  3H      3
  4H      4
  5H      5
  6H      6
  7H      7
  8H      8
  9H      9
  10H    10
  JH     10
  KH     10
  QH     10
  dtype: int64),
 ('S',
  AS      1
  2S      2
  3S      3
  4S      4
  5S      5
  6S      6
  7S      7
  8S      8
  9S      9
  10S    10
  JS     10
  KS     10
  QS     10
  dtype: int64)]

In [65]:
deck.groupby(get_suit, group_keys=False).apply(draw, n=2)

H
H
H
H
H
H
H
H
H
H
H
H
H
S
S
S
S
S
S
S
S
S
S
S
S
S
C
C
C
C
C
C
C
C
C
C
C
C
C
D
D
D
D
D
D
D
D
D
D
D
D
D


AC     1
5C     5
5D     5
7D     7
2H     2
3H     3
KS    10
8S     8
dtype: int64

### Example: Group Weighted Average and Correlation

In [66]:
df = pd.DataFrame({'category': ['a', 'a', 'a', 'a',
                                'b', 'b', 'b', 'b'],
                   'data': np.random.randn(8),
                   'weights': np.random.rand(8)})
df

,category,data,weights
0,a,-0.620521,0.148853
1,a,0.002754,0.078372
2,a,0.968586,0.599105
3,a,-1.129302,0.404450
4,b,-1.138483,0.387785
5,b,2.203645,0.468020
6,b,-0.914722,0.971783
7,b,-0.593495,0.244900


In [67]:
grouped = df.groupby('category')
get_wavg = lambda g: np.average(g['data'], weights=g['weights'])
grouped.apply(get_wavg)

category
a    0.025502
b   -0.214425
dtype: float64

In [68]:
close_px = pd.read_csv('examples/stock_px_2.csv', parse_dates=True,
                       index_col=0)
close_px.info()
close_px[-4:]

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2214 entries, 2003-01-02 to 2011-10-14
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   AAPL    2214 non-null   float64
 1   MSFT    2214 non-null   float64
 2   XOM     2214 non-null   float64
 3   SPX     2214 non-null   float64
dtypes: float64(4)
memory usage: 86.5 KB


,AAPL,MSFT,XOM,SPX
2011-10-11,400.29,27.00,76.27,1195.54
2011-10-12,402.19,26.96,77.16,1207.25
2011-10-13,408.43,27.18,76.37,1203.66
2011-10-14,422.00,27.27,78.11,1224.58


In [69]:
# 计算其他股票和spx的相关性, 返回是一个series

df2013 = close_px[close_px.index.year == 2003]
print(df2013)
df2013.corrwith(df2013['SPX'])

             AAPL   MSFT    XOM      SPX
2003-01-02   7.40  21.11  29.22   909.03
2003-01-03   7.45  21.14  29.24   908.59
2003-01-06   7.45  21.52  29.96   929.01
2003-01-07   7.43  21.93  28.95   922.93
2003-01-08   7.28  21.31  28.83   909.93
...           ...    ...    ...      ...
2003-12-24  10.20  21.44  33.73  1094.04
2003-12-26  10.39  21.57  33.91  1095.89
2003-12-29  10.57  21.77  34.36  1109.48
2003-12-30  10.64  21.82  34.45  1109.64
2003-12-31  10.69  21.70  34.72  1111.92

[252 rows x 4 columns]


AAPL    0.882692
MSFT    0.675932
XOM     0.860104
SPX     1.000000
dtype: float64

In [70]:
# 计算当前元素和前一个元素的变化百分比, 如果没有前一个元素就是na
rets = close_px.pct_change().dropna()
# close_px.pct_change?
rets

,AAPL,MSFT,XOM,SPX
2003-01-03,0.006757,0.001421,0.000684,-0.000484
2003-01-06,0.000000,0.017975,0.024624,0.022474
2003-01-07,-0.002685,0.019052,-0.033712,-0.006545
2003-01-08,-0.020188,-0.028272,-0.004145,-0.014086
2003-01-09,0.008242,0.029094,0.021159,0.019386
...,...,...,...,...
2011-10-10,0.051406,0.026286,0.036977,0.034125
2011-10-11,0.029526,0.002227,-0.000131,0.000544
2011-10-12,0.004747,-0.001481,0.011669,0.009795
2011-10-13,0.015515,0.008160,-0.010238,-0.002974


In [71]:
get_year = lambda x: x.year
by_year = rets.groupby(get_year)
print(type(by_year))

# x是一个dataframe, 返回的是一个series, 内容是其他各个股票和spx的相关性
spx_corr = lambda x: x.corrwith(x['SPX'])

# def spx_corr(x):
#     print(type(x))
#     print(x)
#     return x.corrwith(x['SPX'])

# by_year.apply?
# for name, df in by_year:
#     print(name)
#     print(df)
#     print(df.corrwith(df['SPX']))
#     break

# by_year每一个group是一年的数据的dataframe
# 每一年dataframe通过spx_corr计算的到一个series
# series里面保存了各个股票和spx的相关性
by_year.apply(spx_corr)


<class 'pandas.core.groupby.generic.DataFrameGroupBy'>


,AAPL,MSFT,XOM,SPX
2003,0.541124,0.745174,0.661265,1.0
2004,0.374283,0.588531,0.557742,1.0
2005,0.467540,0.562374,0.631010,1.0
2006,0.428267,0.406126,0.518514,1.0
2007,0.508118,0.658770,0.786264,1.0
2008,0.681434,0.804626,0.828303,1.0
2009,0.707103,0.654902,0.797921,1.0
2010,0.710105,0.730118,0.839057,1.0
2011,0.691931,0.800996,0.859975,1.0


In [72]:
# by_year每一个group是一年的数据的dataframe
# 每一年dataframe通过lambda计算得到一个scala, 
# scala为aapl和msft的相关性
by_year.apply(lambda g: g['AAPL'].corr(g['MSFT']))

2003    0.480868
2004    0.259024
2005    0.300093
2006    0.161735
2007    0.417738
2008    0.611901
2009    0.432738
2010    0.571946
2011    0.581987
dtype: float64

### Example: Group-Wise Linear Regression

In [73]:
import statsmodels.api as sm
def regress(data, yvar, xvars):
    Y = data[yvar]
    X = data[xvars]
    X['intercept'] = 1.
    result = sm.OLS(Y, X).fit()
    return result.params

In [74]:
by_year.apply(regress, 'AAPL', ['SPX'])

,SPX,intercept
2003,1.195406,0.000710
2004,1.363463,0.004201
2005,1.766415,0.003246
2006,1.645496,0.000080
2007,1.198761,0.003438
2008,0.968016,-0.001110
2009,0.879103,0.002954
2010,1.052608,0.001261
2011,0.806605,0.001514


## Pivot Tables and Cross-Tabulation

In [75]:
tips.pivot_table?

Signature:
tips.pivot_table(
    values=None,
    index=None,
    columns=None,
    aggfunc='mean',
    fill_value=None,
    margins=False,
    dropna=True,
    margins_name='All',
    observed=False,
) -> 'DataFrame'
Docstring:
Create a spreadsheet-style pivot table as a DataFrame.

The levels in the pivot table will be stored in MultiIndex objects
(hierarchical indexes) on the index and columns of the result DataFrame.

Parameters
----------
values : column to aggregate, optional
index : column, Grouper, array, or list of the previous
    If an array is passed, it must be the same length as the data. The
    list can contain any of the other types (except list).
    Keys to group by on the pivot table index.  If an array is passed,
    it is being used as the same manner as column values.
columns : column, Grouper, array, or list of the previous
    If an array is passed, it must be the same length as the data. The
    list can contain any of the other types (except list).
    Keys t

In [76]:
df = pd.DataFrame({"A": ["foo", "foo", "foo", "foo", "foo",
                         "bar", "bar", "bar", "bar"],
                   "B": ["one", "one", "one", "two", "two",
                         "one", "one", "two", "two"],
                   "C": ["small", "large", "large", "small",
                         "small", "large", "small", "small",
                         "large"],
                   "D": [1, 2, 2, 3, 3, 4, 5, 6, 7],
                   "E": [2, 4, 5, 5, 6, 6, 8, 9, 9]})
df

,A,B,C,D,E
0,foo,one,small,1,2
1,foo,one,large,2,4
2,foo,one,large,2,5
3,foo,two,small,3,5
4,foo,two,small,3,6
5,bar,one,large,4,6
6,bar,one,small,5,8
7,bar,two,small,6,9
8,bar,two,large,7,9


In [77]:
'''
pivot_table 将一张表进行汇聚操作, index指明哪些列的值压缩成index, 
column指明那些列作为column(逆时针旋转和压缩)

具体操作过程是：
1. 对指定为index的所有列进行压缩，形成新的行键
2. 对指定为column的所有列进行逆时针旋转和压缩，形成新的列键
3. 对每个新的行/列键，使用汇聚函数计算值
'''

'''A,B的值作为index, C的值作为column, values为被汇聚的列'''
df.pivot_table(values='D', index='A', columns='C', aggfunc=np.sum)


C,large,small
A,,
bar,11,11
foo,4,7


In [78]:
'''A的值作为index, value为被汇聚的列, 如果没有column, 这直接对value整列汇聚'''
df.pivot_table(values='D', index='A', aggfunc=np.sum)

,D
A,
bar,22
foo,11


In [79]:
'''C的值作为列键, value为被汇聚的列, 如果没有index, 则只有一行'''
df.pivot_table(values='D', columns='C', aggfunc=np.sum)

C,large,small
D,15,18


In [80]:
df.pivot_table(values='D', aggfunc=np.sum)

ValueError: No group keys passed!

In [100]:
# A的值作为index, value为被汇聚的列, 如果没有column, 这直接对value整列汇聚
df.pivot_table(values='D', index=['A', 'B'], aggfunc=np.sum)

D
A   B      
bar one   9
    two  13
foo one   5
    two   6

In [102]:
# # A，B的值作为index, 如果没有column, 这直接对value整列汇聚, 如果没有column, 则对所有能汇聚的列进行汇聚
df.pivot_table(index=['A', 'B'], aggfunc=np.sum)

D   E
A   B          
bar one   9  14
    two  13  18
foo one   5  11
    two   6  11

In [94]:

df.pivot_table(values='D', index=['A', 'B'], columns='C', aggfunc=np.sum)

C        large  small
A   B                
bar one    4.0    5.0
    two    7.0    6.0
foo one    4.0    1.0
    two    NaN    6.0

In [106]:
df.pivot_table(values='D', index='A', columns=['B', 'C'], aggfunc=np.sum)

B     one         two      
C   large small large small
A                          
bar   4.0   5.0   7.0   6.0
foo   4.0   1.0   NaN   6.0

In [81]:
df.pivot_table(values='E', index=['A', 'B'], columns=['C', 'D'], aggfunc=np.sum)

C       large           small                
D           2    4    7     1     3    5    6
A   B                                        
bar one   NaN  6.0  NaN   NaN   NaN  8.0  NaN
    two   NaN  NaN  9.0   NaN   NaN  NaN  9.0
foo one   9.0  NaN  NaN   2.0   NaN  NaN  NaN
    two   NaN  NaN  NaN   NaN  11.0  NaN  NaN

In [84]:
table = pd.pivot_table(df, values=['D', 'E'], index=['A', 'B'], columns='C',
                    aggfunc={'D': np.mean,
                             'E': np.mean})
table

D           E      
C       large small large small
A   B                          
bar one   4.0   5.0   6.0   8.0
    two   7.0   6.0   9.0   9.0
foo one   2.0   1.0   4.5   2.0
    two   NaN   3.0   NaN   5.5

In [89]:
print(tips)
tips.pivot_table(index=['day', 'smoker'])

     total_bill   tip smoker   day    time  size   tip_pct
0         16.99  1.01     No   Sun  Dinner     2  0.059447
1         10.34  1.66     No   Sun  Dinner     3  0.160542
2         21.01  3.50     No   Sun  Dinner     3  0.166587
3         23.68  3.31     No   Sun  Dinner     2  0.139780
4         24.59  3.61     No   Sun  Dinner     4  0.146808
..          ...   ...    ...   ...     ...   ...       ...
239       29.03  5.92     No   Sat  Dinner     3  0.203927
240       27.18  2.00    Yes   Sat  Dinner     2  0.073584
241       22.67  2.00    Yes   Sat  Dinner     2  0.088222
242       17.82  1.75     No   Sat  Dinner     2  0.098204
243       18.78  3.00     No  Thur  Dinner     2  0.159744

[244 rows x 7 columns]


size       tip   tip_pct  total_bill
day  smoker                                          
Fri  No      2.250000  2.812500  0.151650   18.420000
     Yes     2.066667  2.714000  0.174783   16.813333
Sat  No      2.555556  3.102889  0.158048   19.661778
     Yes     2.476190  2.875476  0.147906   21.276667
Sun  No      2.929825  3.167895  0.160113   20.506667
     Yes     2.578947  3.516842  0.187250   24.120000
Thur No      2.488889  2.673778  0.160298   17.113111
     Yes     2.352941  3.030000  0.163863   19.190588

In [85]:
tips.pivot_table(['tip_pct', 'size'], index=['time', 'day'],
                 columns='smoker')

size             tip_pct          
smoker             No       Yes        No       Yes
time   day                                         
Dinner Fri   2.000000  2.222222  0.139622  0.165347
       Sat   2.555556  2.476190  0.158048  0.147906
       Sun   2.929825  2.578947  0.160113  0.187250
       Thur  2.000000       NaN  0.159744       NaN
Lunch  Fri   3.000000  1.833333  0.187735  0.188937
       Thur  2.500000  2.352941  0.160311  0.163863

In [86]:
tips.pivot_table(['tip_pct', 'size'], index=['time', 'day'],
                 columns='smoker', margins=True)

size                       tip_pct                    
smoker             No       Yes       All        No       Yes       All
time   day                                                             
Dinner Fri   2.000000  2.222222  2.166667  0.139622  0.165347  0.158916
       Sat   2.555556  2.476190  2.517241  0.158048  0.147906  0.153152
       Sun   2.929825  2.578947  2.842105  0.160113  0.187250  0.166897
       Thur  2.000000       NaN  2.000000  0.159744       NaN  0.159744
Lunch  Fri   3.000000  1.833333  2.000000  0.187735  0.188937  0.188765
       Thur  2.500000  2.352941  2.459016  0.160311  0.163863  0.161301
All          2.668874  2.408602  2.569672  0.159328  0.163196  0.160803

In [87]:
tips.pivot_table('tip_pct', index=['time', 'smoker'], columns='day',
                 aggfunc=len, margins=True)

day             Fri   Sat   Sun  Thur    All
time   smoker                               
Dinner No       3.0  45.0  57.0   1.0  106.0
       Yes      9.0  42.0  19.0   NaN   70.0
Lunch  No       1.0   NaN   NaN  44.0   45.0
       Yes      6.0   NaN   NaN  17.0   23.0
All            19.0  87.0  76.0  62.0  244.0

In [ ]:
tips.pivot_table('tip_pct', index=['time', 'size', 'smoker'],
                 columns='day', aggfunc='mean', fill_value=0)

### Cross-Tabulations: Crosstab

In [ ]:
from io import StringIO
data = """\
Sample  Nationality  Handedness
1   USA  Right-handed
2   Japan    Left-handed
3   USA  Right-handed
4   Japan    Right-handed
5   Japan    Left-handed
6   Japan    Right-handed
7   USA  Right-handed
8   USA  Left-handed
9   Japan    Right-handed
10  USA  Right-handed"""
data = pd.read_table(StringIO(data), sep='\s+')

In [ ]:
data

In [ ]:
pd.crosstab(data.Nationality, data.Handedness, margins=True)

In [ ]:
pd.crosstab([tips.time, tips.day], tips.smoker, margins=True)

In [ ]:
pd.options.display.max_rows = PREVIOUS_MAX_ROWS

## Conclusion